# 2017 NBA Players Dataset Exploration

In this practice case, we will do some exploration on 2017 NBA Players dataset. The original dataset consists of records from 1950 season all the way to 2017 season. We will answer some key questions.

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import preprocessing

In [2]:
#load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/Syukrondzeko/Fellowship-5/main/Seasons_Stats.csv', index_col=0)

In [3]:
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN,NaN,144.0,516.0,0.279,NaN,NaN,NaN,144.0,516.0,0.279,0.279,170.0,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN,NaN,102.0,274.0,0.372,NaN,NaN,NaN,102.0,274.0,0.372,0.372,75.0,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN,NaN,174.0,499.0,0.349,NaN,NaN,NaN,174.0,499.0,0.349,0.349,90.0,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,22.0,86.0,0.256,NaN,NaN,NaN,22.0,86.0,0.256,0.256,19.0,34.0,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,21.0,82.0,0.256,NaN,NaN,NaN,21.0,82.0,0.256,0.256,17.0,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


Since we will only explore records on 2017 season, we select only rows from 2017 season.

In [4]:
#Make a subset of 2017 season
df17 = df[df['Year'] == 2017]

In [5]:
df17.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,0.560,0.724,0.144,1.9,7.1,4.5,5.5,1.7,0.6,8.3,15.9,NaN,1.2,0.9,2.1,0.095,NaN,-0.3,-2.2,-2.5,-0.1,134.0,341.0,0.393,94.0,247.0,0.381,40.0,94.0,0.426,0.531,44.0,49.0,0.898,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0
24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,0.565,0.529,0.353,3.9,18.0,11.0,4.9,1.2,2.0,9.7,16.8,NaN,0.5,0.5,0.9,0.082,NaN,-1.8,-1.2,-3.0,-0.1,70.0,170.0,0.412,37.0,90.0,0.411,33.0,80.0,0.413,0.521,45.0,60.0,0.750,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0
24098,2017.0,Quincy Acy,PF,26.0,DAL,6.0,0.0,48.0,-1.4,0.355,0.412,0.176,4.6,15.2,9.7,0.0,0.0,0.0,9.8,20.0,NaN,-0.2,0.0,-0.1,-0.133,NaN,-10.1,-6.0,-16.2,-0.2,5.0,17.0,0.294,1.0,7.0,0.143,4.0,10.0,0.400,0.324,2.0,3.0,0.667,2.0,6.0,8.0,0.0,0.0,0.0,2.0,9.0,13.0
24099,2017.0,Quincy Acy,PF,26.0,BRK,32.0,1.0,510.0,13.1,0.587,0.542,0.373,3.8,18.2,11.1,5.4,1.3,2.2,9.6,16.5,NaN,0.6,0.5,1.1,0.102,NaN,-1.1,-0.7,-1.8,0.0,65.0,153.0,0.425,36.0,83.0,0.434,29.0,70.0,0.414,0.542,43.0,57.0,0.754,18.0,89.0,107.0,18.0,14.0,15.0,19.0,58.0,209.0
24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,0.589,0.002,0.392,13.0,15.5,14.2,5.4,1.8,2.6,16.0,16.2,NaN,3.3,3.1,6.4,0.130,NaN,-0.7,1.2,0.5,1.5,374.0,655.0,0.571,0.0,1.0,0.000,374.0,654.0,0.572,0.571,157.0,257.0,0.611,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0


The NBA allows for in-season player trade, hence there are players who play for multiple teams in a season. Those players would then have multiple entries on this dataset. To simplify our work, we will only use the first row for each of those players. To do that, we use dataframe.drop_duplicates()

In [6]:
#Drop duplicates (players who switch team in-season)
df17.drop_duplicates(subset='Player', inplace=True)
df17 = df17.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df17.head()

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,0.560,0.724,0.144,1.9,7.1,4.5,5.5,1.7,0.6,8.3,15.9,NaN,1.2,0.9,2.1,0.095,NaN,-0.3,-2.2,-2.5,-0.1,134.0,341.0,0.393,94.0,247.0,0.381,40.0,94.0,0.426,0.531,44.0,49.0,0.898,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0
1,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,0.565,0.529,0.353,3.9,18.0,11.0,4.9,1.2,2.0,9.7,16.8,NaN,0.5,0.5,0.9,0.082,NaN,-1.8,-1.2,-3.0,-0.1,70.0,170.0,0.412,37.0,90.0,0.411,33.0,80.0,0.413,0.521,45.0,60.0,0.750,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0
2,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,0.589,0.002,0.392,13.0,15.5,14.2,5.4,1.8,2.6,16.0,16.2,NaN,3.3,3.1,6.4,0.130,NaN,-0.7,1.2,0.5,1.5,374.0,655.0,0.571,0.0,1.0,0.000,374.0,654.0,0.572,0.571,157.0,257.0,0.611,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0
3,24101,2017.0,Arron Afflalo,SG,31.0,SAC,61.0,45.0,1580.0,9.0,0.559,0.360,0.221,0.7,8.4,4.6,7.4,0.7,0.4,8.4,14.4,NaN,1.2,0.2,1.4,0.043,NaN,-1.4,-2.0,-3.5,-0.6,185.0,420.0,0.440,62.0,151.0,0.411,123.0,269.0,0.457,0.514,83.0,93.0,0.892,9.0,116.0,125.0,78.0,21.0,7.0,42.0,104.0,515.0
4,24102,2017.0,Alexis Ajinca,C,28.0,NOP,39.0,15.0,584.0,12.9,0.529,0.022,0.225,8.3,23.8,16.0,3.1,1.7,3.1,13.7,17.2,NaN,0.0,0.9,1.0,0.080,NaN,-5.1,1.0,-4.1,-0.3,89.0,178.0,0.500,0.0,4.0,0.000,89.0,174.0,0.511,0.500,29.0,40.0,0.725,46.0,131.0,177.0,12.0,20.0,22.0,31.0,77.0,207.0


Note: Players who play for multiple teams have multiple entries: one for each team, and one more row with the team column is filled with ‘TOT’. The “TOT rows” are to show the total (combined) stats for those players from their multiple teams. The dataframe.drop_duplicates() function only leaves the “TOT rows” for those players, and dropping the real teams rows. To simplify our work, we will treat ‘TOT’ as a separate team. Thus, all players who switch team mid-season will be treated as player of TOT team. It also implies that there are 31 teams in this exploration, instead of 30 that NBA actually has.

In [8]:
df17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 53 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   486 non-null    int64  
 1   Year    486 non-null    float64
 2   Player  486 non-null    object 
 3   Pos     486 non-null    object 
 4   Age     486 non-null    float64
 5   Tm      486 non-null    object 
 6   G       486 non-null    float64
 7   GS      486 non-null    float64
 8   MP      486 non-null    float64
 9   PER     486 non-null    float64
 10  TS%     485 non-null    float64
 11  3PAr    485 non-null    float64
 12  FTr     485 non-null    float64
 13  ORB%    486 non-null    float64
 14  DRB%    486 non-null    float64
 15  TRB%    486 non-null    float64
 16  AST%    486 non-null    float64
 17  STL%    486 non-null    float64
 18  BLK%    486 non-null    float64
 19  TOV%    485 non-null    float64
 20  USG%    486 non-null    float64
 21  blanl   0 non-null      float64
 22  OW

In [9]:
#Drop 2 blank columns
df17.drop(['blanl','blank2'],axis=1,inplace=True)

In [10]:
df17.head()

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,0.560,0.724,0.144,1.9,7.1,4.5,5.5,1.7,0.6,8.3,15.9,1.2,0.9,2.1,0.095,-0.3,-2.2,-2.5,-0.1,134.0,341.0,0.393,94.0,247.0,0.381,40.0,94.0,0.426,0.531,44.0,49.0,0.898,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0
1,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,0.565,0.529,0.353,3.9,18.0,11.0,4.9,1.2,2.0,9.7,16.8,0.5,0.5,0.9,0.082,-1.8,-1.2,-3.0,-0.1,70.0,170.0,0.412,37.0,90.0,0.411,33.0,80.0,0.413,0.521,45.0,60.0,0.750,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0
2,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,0.589,0.002,0.392,13.0,15.5,14.2,5.4,1.8,2.6,16.0,16.2,3.3,3.1,6.4,0.130,-0.7,1.2,0.5,1.5,374.0,655.0,0.571,0.0,1.0,0.000,374.0,654.0,0.572,0.571,157.0,257.0,0.611,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0
3,24101,2017.0,Arron Afflalo,SG,31.0,SAC,61.0,45.0,1580.0,9.0,0.559,0.360,0.221,0.7,8.4,4.6,7.4,0.7,0.4,8.4,14.4,1.2,0.2,1.4,0.043,-1.4,-2.0,-3.5,-0.6,185.0,420.0,0.440,62.0,151.0,0.411,123.0,269.0,0.457,0.514,83.0,93.0,0.892,9.0,116.0,125.0,78.0,21.0,7.0,42.0,104.0,515.0
4,24102,2017.0,Alexis Ajinca,C,28.0,NOP,39.0,15.0,584.0,12.9,0.529,0.022,0.225,8.3,23.8,16.0,3.1,1.7,3.1,13.7,17.2,0.0,0.9,1.0,0.080,-5.1,1.0,-4.1,-0.3,89.0,178.0,0.500,0.0,4.0,0.000,89.0,174.0,0.511,0.500,29.0,40.0,0.725,46.0,131.0,177.0,12.0,20.0,22.0,31.0,77.0,207.0


# 1. Youngest and oldest players of each teams

In [11]:
#Sort the values from oldest to youngest. The drop_duplicates will drop non-oldest players for every team.
oldest = df17.sort_values('Age', ascending=False).drop_duplicates(['Tm'])
df_oldest = oldest[['Tm','Player','Pos','Age']].sort_values('Tm').reset_index()
df_oldest.drop(['index'], axis=1, inplace=True)

#Sort the values from youngest to oldest. The drop_duplicates will drop non-youngest players for every team.
youngest = df17.sort_values('Age', ascending=True).drop_duplicates(['Tm'])
df_youngest = youngest[['Tm','Player','Pos','Age']].sort_values('Tm').reset_index()
df_youngest.drop(['index'], axis=1, inplace=True)

#Merge the oldest and youngest players dataframe using inner join.
old_young = pd.merge(df_oldest, df_youngest, on='Tm', how='inner')
old_young.columns = [['Tm', 'Oldest Player', 'Pos', 'Oldest Age', 'Youngest Player', 'Pos', 'Youngest Age']]
old_young

,Tm,Oldest Player,Pos,Oldest Age,Youngest Player,Pos,Youngest Age
0,ATL,Gary Neal,SG,32.0,Taurean Waller-Prince,SF,22.0
1,BOS,Gerald Green,SF,31.0,Jaylen Brown,SF,20.0
2,BRK,Luis Scola,PF,36.0,Isaiah Whitehead,PG,21.0
3,CHI,Dwyane Wade,SG,35.0,Bobby Portis,PF,21.0
4,CHO,Brian Roberts,PG,31.0,Christian Wood,PF,21.0
5,CLE,Chris Andersen,C,38.0,Kay Felder,PG,21.0
6,DAL,Dirk Nowitzki,PF,38.0,Ben Bentil,PF,21.0
7,DEN,Mike Miller,SF,36.0,Jamal Murray,SG,19.0
8,DET,Beno Udrih,PG,34.0,Stanley Johnson,SF,20.0
9,GSW,David West,C,36.0,Kevon Looney,C,20.0


# 2. Players with most minute-played per position

In [12]:
#Sort the values from most minute-played. The drop_duplicates will drop non-most-playing players for every position.
df_mostplay = df17.sort_values('MP', ascending=False).drop_duplicates(['Pos'])
df_mostplay[['Pos','Player','Tm','MP']].sort_values('Pos')

,Pos,Player,Tm,MP
432,C,Karl-Anthony Towns,MIN,3030.0
27,PF,Harrison Barnes,DAL,2803.0
253,PF-C,Joffrey Lauvergne,TOT,980.0
171,PG,James Harden,HOU,2947.0
461,SF,Andrew Wiggins,MIN,3048.0
287,SG,C.J. McCollum,POR,2796.0


# 3. Team with best stats

Best total rebound percentage

In [13]:
#Group by team, find the mean of TRB%, sort the value from the highest, show the top row.
df17.groupby('Tm')['TRB%'].mean().sort_values(ascending=False).head(1)

Tm
WAS    13.45
Name: TRB%, dtype: float64

Best assist percentage

In [14]:
#Group by team, find the mean of AST%, sort the value from the highest, show the top row.
df17.groupby('Tm')['AST%'].mean().sort_values(ascending=False).head(1)

Tm
DEN    15.86
Name: AST%, dtype: float64

Best steal percentage

In [15]:
#Group by team, find the mean of STL%, sort the value from the highest, show the top row.
df17.groupby('Tm')['STL%'].mean().sort_values(ascending=False).head(1)

Tm
MIN    2.371429
Name: STL%, dtype: float64

Best block percentage

In [16]:
#Group by team, find the mean of BLK%, sort the value from the highest, show the top row.
df17.groupby('Tm')['BLK%'].mean().sort_values(ascending=False).head(1)

Tm
GSW    2.74
Name: BLK%, dtype: float64

# 4. Player of the Year

To find out the best player of 2017 season, we use the following formula:

Ovr = 2*TS% + 2*AST% + 1.5*TRB% + STL% + BLK% + MP

where:
Ovr = overall score

TS% = True shooting percentage; a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.

AST% = Assist percentage

TRB% = Total rebound percentage

STL% = Steal percentage

BLK% = Block percentage

MP = minutes played

Note: all features are normalized using MinMaxScaler before being plugged-in to the overall score calculation.

Why are those variables selected to calculate player’s overall score? Here are some reasonings. TS% and AST% are given top priority, because basketball is ultimately about scoring as many as possible, so the efficiency of shooting and assist should be priority. TRB% is given second highest weight because rebound is a component of triple-double. STL% and BLK% are included to measure player’s defensive performance. MP is included because best players need to contribute as many playing time as possible to their team. It is also to make sure that the high scoring percentage (and other percentages) are maintained consistently throughout a long period of time.


In [17]:
#Total scoring percentage values are written in different form compard to other percentage stats, so we adjust the format
df17['TS%'] = df17['TS%']*100

In [18]:
#Fill missing value with 0.
x = df17[['TS%','AST%','TRB%','STL%','BLK%','MP']].fillna(0)

#Apply MinMaxScaler to said features
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df17_ovr_norm = pd.DataFrame(x_scaled, columns=('TS%','AST%','TRB%','STL%','BLK%','MP'))

df17_ovr_norm

,TS%,AST%,TRB%,STL%,BLK%,MP
0,0.700876,0.095986,0.079787,0.153153,0.034682,0.345914
1,0.707134,0.085515,0.195035,0.108108,0.115607,0.182803
2,0.737171,0.094241,0.251773,0.162162,0.150289,0.783722
3,0.699625,0.129145,0.081560,0.063063,0.023121,0.518215
4,0.662078,0.054101,0.283688,0.153153,0.179191,0.191336
...,...,...,...,...,...,...
481,0.755945,0.158813,0.228723,0.162162,0.173410,0.565802
482,0.635795,0.212914,0.234043,0.063063,0.190751,0.171972
483,0.433041,0.092496,0.312057,0.081081,0.213873,0.035117
484,0.629537,0.106457,0.141844,0.081081,0.086705,0.276337


In [19]:
#Adding player, team, and position columns to the calculation dataframe
df17_ovr_norm['Player'] = df17['Player']
df17_ovr_norm['Tm'] = df17['Tm']
df17_ovr_norm['Pos'] = df17['Pos']
df17_ovr_norm = df17_ovr_norm[['Player','Tm','Pos','TS%','AST%','TRB%','STL%','BLK%','MP']]

#Perform the overall rating calculation
df17_ovr_norm['Ovr'] = 2*df17_ovr_norm['TS%'] + 2*df17_ovr_norm['AST%'] \
                            + 1.5*df17_ovr_norm['TRB%'] + df17_ovr_norm['STL%']\
                            + df17_ovr_norm['BLK%'] + df17_ovr_norm['MP']
df17_ovr_norm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Player,Tm,Pos,TS%,AST%,TRB%,STL%,BLK%,MP,Ovr
0,Alex Abrines,OKC,SG,0.700876,0.095986,0.079787,0.153153,0.034682,0.345914,2.247154
1,Quincy Acy,TOT,PF,0.707134,0.085515,0.195035,0.108108,0.115607,0.182803,2.284368
2,Steven Adams,OKC,C,0.737171,0.094241,0.251773,0.162162,0.150289,0.783722,3.136657
3,Arron Afflalo,SAC,SG,0.699625,0.129145,0.081560,0.063063,0.023121,0.518215,2.384278
4,Alexis Ajinca,NOP,C,0.662078,0.054101,0.283688,0.153153,0.179191,0.191336,2.381569
...,...,...,...,...,...,...,...,...,...,...
481,Cody Zeller,CHO,PF,0.755945,0.158813,0.228723,0.162162,0.173410,0.565802,3.073976
482,Tyler Zeller,BOS,C,0.635795,0.212914,0.234043,0.063063,0.190751,0.171972,2.474269
483,Stephen Zimmerman,ORL,C,0.433041,0.092496,0.312057,0.081081,0.213873,0.035117,1.849229
484,Paul Zipser,CHI,SF,0.629537,0.106457,0.141844,0.081081,0.086705,0.276337,2.128878


So, without further ado, 2017 NBA Player of the Year according to our scoring system is... (drum roll please)

In [20]:
#Find out the best player of 2017 according to our calculation
df17_ovr_norm[df17_ovr_norm['Ovr'] == df17_ovr_norm['Ovr'].max()]

,Player,Tm,Pos,TS%,AST%,TRB%,STL%,BLK%,MP,Ovr
457,Russell Westbrook,OKC,PG,0.693367,1.0,0.303191,0.207207,0.052023,0.919265,5.020016


In [21]:
df17[df17['Player'] == 'Russell Westbrook']

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
457,24654,2017.0,Russell Westbrook,PG,28.0,OKC,81.0,81.0,2802.0,30.6,55.4,0.3,0.433,5.4,28.8,17.1,57.3,2.3,0.9,15.9,41.7,8.5,4.6,13.1,0.224,10.9,4.7,15.6,12.4,824.0,1941.0,0.425,200.0,583.0,0.343,624.0,1358.0,0.459,0.476,710.0,840.0,0.845,137.0,727.0,864.0,840.0,133.0,31.0,438.0,190.0,2558.0


# 5. Team of the Year
We use the same formula from previous section and find the average for entire teams.

In [22]:
#Group by team, find the mean of Ovr, sort the value from the highest, show the top five row.
df17_ovr_norm.groupby('Tm').mean('Ovr').sort_values('Ovr',ascending=False).head()

,TS%,AST%,TRB%,STL%,BLK%,MP,Ovr
Tm,,,,,,,
GSW,0.712307,0.243630,0.202600,0.156156,0.158382,0.422470,2.952783
DEN,0.709136,0.276789,0.187943,0.169369,0.055106,0.399190,2.877431
SAS,0.700954,0.260035,0.171764,0.136261,0.110188,0.405912,2.831985
BOS,0.701293,0.254334,0.183688,0.123123,0.086320,0.431878,2.828108
HOU,0.699047,0.223788,0.173077,0.148302,0.092486,0.447022,2.793096


The champ, Golden State Warriors rank highest on our scoring system.